In [1]:
url = ''
download_dir = ''
tor_password= ''

In [2]:
# !brew install tor
# !pip install 'requests[socks]'
# !pip install pysocks
# !pip install speedtest-cli

# You must start TOR

In [3]:
# !sudo vim ~/Library/Application\ Support/TorBrowser-Data/Tor/torrc
#OR sudo vim /opt/homebrew/etc/tor/torrc
# netstat -an | grep 9050
# sudo chmod 644 /opt/homebrew/etc/tor/torrc
# !tor


#SocksPort 9050
#ControlPort 9051
#HashedControlPassword your_hashed_password

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import sys
from stem import Signal
from stem.control import Controller
import threading

# Function to print in-place logging with thread safety
log_lock = threading.Lock()  # Lock for thread-safe logging
def log_message(message):
    with log_lock:
        sys.stdout.write(f"{message}\n")
        sys.stdout.flush()

# Function to check and switch Tor identity (new node)
def switch_tor_identity():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate(password=tor_password)  # Set the password for Tor's control port
        controller.signal(Signal.NEWNYM)  # Request a new identity
        time.sleep(5)  # Wait for Tor to establish the new connection
        log_message("Switched to a new Tor identity.")

# Function to download a file with retries
def download_file(file_url, file_name, proxies, headers, retries=3):
    # Uncomment this block if you don't want to force re-download for testing purposes
    if os.path.exists(file_name):
        log_message(f"File {file_name} already exists. Skipping download.")
        return  # Skip if file already exists

    attempt = 0  # Number of attempts for retries
    start_time = time.time()

    while attempt < retries:
        try:
            with requests.get(file_url, proxies=proxies, headers=headers, stream=True, timeout=300) as file_response:
                if file_response.status_code == 200:
                    total_length = int(file_response.headers.get('content-length', 0))
                    downloaded = 0
                    chunk_size = 1024 * 1024  # 1 MB chunks
                    last_logged_percentage = 0

                    with open(file_name, 'wb') as f:
                        for data in file_response.iter_content(chunk_size=chunk_size):
                            f.write(data)
                            downloaded += len(data)

                            # Calculate percentage downloaded and log every 10% progress
                            if total_length > 0:
                                percentage_downloaded = int((downloaded / total_length) * 100)
                                if percentage_downloaded >= last_logged_percentage + 10:  # Log every 10%
                                    log_message(f"Downloading {file_name}: {percentage_downloaded}% complete.")
                                    last_logged_percentage = percentage_downloaded

                    # Calculate and log the total download time and average speed
                    total_time = time.time() - start_time
                    avg_speed_mbps = (downloaded / total_time) / 1_000_000  # Convert to Mbps
                    log_message(f"Download complete: {file_name}. Average speed: {avg_speed_mbps:.2f} Mbps")
                    return avg_speed_mbps
                else:
                    log_message(f"Failed to download: {file_url}. Status code: {file_response.status_code}")
                    return 0
        except requests.exceptions.RequestException as e:
            attempt += 1
            log_message(f"Error downloading {file_url}. Retrying... (Attempt {attempt}/{retries}) Error: {str(e)}")
            time.sleep(5)  # Wait before retrying

    log_message(f"Failed to download {file_url} after {retries} attempts.")
    return 0

# Main function
def main():
    url = 'http://x6zdxw6vt3gtpv35yqloydttvfvwyrju3opkmp4xejmlfxto7ahgnpyd.onion/CIIG/'
    download_dir = '/Volumes/Untitled/kim_customer'

    try:
        max_threads = 3
        log_message(f"Max threads based on speed: {max_threads}")

        # Set up Tor proxies
        proxies = {
            'http': 'socks5h://127.0.0.1:9050',
            'https': 'socks5h://127.0.0.1:9050'
        }

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
            log_message(f"Created download directory: {download_dir}")

        # Test Tor connection
        with requests.Session() as session:
            session.proxies = proxies
            session.headers.update(headers)

            test_url = "http://check.torproject.org"
            try:
                response = session.get(test_url)
                if 'Congratulations' in response.text:
                    log_message("Tor connection is successful.")
                else:
                    log_message("Tor connection failed.")
            except Exception as e:
                log_message(f"Error checking Tor connection: {e}")

            # Request the page via the Tor network
            log_message(f"Requesting page: {url}")
            response = session.get(url)

            # Check if request succeeded
            if response.status_code == 200:
                log_message(f"Successfully accessed the page. Status code: {response.status_code}")
                soup = BeautifulSoup(response.text, 'html.parser')
                links = soup.find_all('a')
                log_message(f"Found {len(links)} links on the page.")

                # Use ThreadPoolExecutor to download files in parallel with retries
                with ThreadPoolExecutor(max_workers=max_threads) as executor:
                    futures = []
                    for link in links:
                        file_url = url + link.get('href')
                        file_name = os.path.join(download_dir, link.get('href').split('/')[-1])

                        # Uncomment this block if you don't want to force re-download
                        if os.path.exists(file_name):
                            log_message(f"File {file_name} already exists. Skipping download.")
                            continue

                        switch_tor_identity()  # Switch Tor node before each download
                        futures.append(executor.submit(download_file, file_url, file_name, proxies, headers))

                    # Wait for all futures to complete
                    for future in as_completed(futures):
                        avg_speed = future.result()
                        if avg_speed:
                            log_message(f"Average download speed for the file: {avg_speed:.2f} Mbps")
            else:
                log_message(f"Failed to access the page. Status code: {response.status_code}")
    except Exception as e:
        log_message(f"Error in main function: {e}")

if __name__ == '__main__':
    main()